# Wellahh Food Recommendation System

Reference: T. N. Trang Tran, M. Atas, A. Felfernig, and M. Stettinger, “An overview of recommender systems in the healthy food domain,” J. Intell. Inf. Syst., vol. 50, no. 3, pp. 501–526, Jun. 2018, doi: 10.1007/s10844-017-0469-0.

- download dulu datasetnya bang: https://drive.google.com/uc?export=download&id=1zPr5XKMSAeaeY6IKQWnABYmwzlYUBNhU

In [83]:
import urllib
import pandas as pd
import pandas as pd
import numpy as np

from sklearn import preprocessing

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import re
import string
from pathlib import Path
from nltk.stem.snowball import SnowballStemmer

import pickle
import random


# my_file_2 = Path("./recipes.csv")
# if my_file_2.is_file() == False:
#     url = "https://drive.google.com/uc?export=download&id=1zPr5XKMSAeaeY6IKQWnABYmwzlYUBNhU"
#     path = "recipes.csv"
#     urllib.request.urlretrieve(url, path)
path2 = "./recipes.csv"



df2 = pd.read_csv(path2)


# Data Prep

In [84]:
df2.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [85]:
df2 = df2.drop_duplicates(subset=['Name'], keep="last")


In [86]:
df2= df2.reset_index(drop=True)

In [87]:
df2 = df2.drop([idx for idx in range(10000, len(df2))])

In [88]:
df2= df2.reset_index(drop=True)

In [89]:
df2.to_csv('recipes_sedikit.csv')


In [90]:
for idx,row in df2.iterrows():
     df2.iloc[idx:idx+1, 13:14] = row["RecipeIngredientParts"] .replace('c(', '').replace(')', '').replace('"', '').replace(',', '')


In [91]:
df2["RecipeIngredientParts"]

0       blueberries granulated sugar vanilla yogurt le...
1       extra firm tofu eggplant zucchini mushrooms so...
2       graham cracker crumbs sugar butter sugar corns...
3       chicken butter flour milk celery button mushro...
4       sugar margarine egg flour salt buttermilk grah...
                              ...                        
9995    margarine chocolate chips mini marshmallows co...
9996       heavy cream unsalted butter white sugar cognac
9997                                      white chocolate
9998    shrimp plain yogurt fresh lemon juice cayenne ...
9999    napa cabbage olive oil onion soy sauce fresh l...
Name: RecipeIngredientParts, Length: 10000, dtype: object

# Content Based Filtering
- merekomendasikan makanan yang relevan dengan beberapa makanan favorit pengguna,
- relevansi berdasarkan ingredient makanan, nama makanan, dan category makanan


In [92]:

def create_features(x):
    return x['RecipeIngredientParts'] + " " + x['RecipeCategory'] + " " + x["Name"]
    

In [94]:
df2['soup'] = df2.apply(create_features, axis=1)

In [95]:
count = CountVectorizer()
count_matrix = count.fit_transform(df2['soup'])

In [96]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)


In [97]:
df2 = df2.reset_index(drop=True)


In [98]:
def name_tf_idf(x):
    return  x['Name'] 
    

In [99]:
df2['soup_name'] = df2.apply(name_tf_idf, axis=1)

In [100]:
df2.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,soup,soup_name
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan...",blueberries granulated sugar vanilla yogurt le...,Low-Fat Berry Blue Frozen Dessert
1,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc...",extra firm tofu eggplant zucchini mushrooms so...,Carina's Tofu-Vegetable Kebabs
2,43,Best Blackbottom Pie,34879,Barefoot Beachcomber,PT2H,PT20M,PT2H20M,1999-08-21T10:35:00Z,Make and share this Best Blackbottom Pie recip...,character(0),...,267.6,58.0,1.8,42.5,7.0,8.0,1 9-inch pie,"c(""Graham Cracker Crust: In small bowl, combin...",graham cracker crumbs sugar butter sugar corns...,Best Blackbottom Pie
3,44,Warm Chicken A La King,1596,Joan Edington,PT3M,PT35M,PT38M,1999-09-17T04:47:00Z,I copied this one out of a friend's book so ma...,"""https://img.sndimg.com/food/image/upload/w_55...",...,557.2,29.1,3.1,5.0,45.3,2.0,NaN,"c(""Melt 1 1/2 ozs butter, add the flour and co...",chicken butter flour milk celery button mushro...,Warm Chicken A La King
4,45,Buttermilk Pie With Gingersnap Crumb Crust,1580,tristitia,PT50M,PT30M,PT1H20M,1999-08-06T00:40:00Z,Make and share this Buttermilk Pie With Ginger...,"""https://img.sndimg.com/food/image/upload/w_55...",...,281.8,37.5,0.5,24.7,4.2,8.0,NaN,"c(""Preheat oven to 350°F."", ""Make pie crust, u...",sugar margarine egg flour salt buttermilk grah...,Buttermilk Pie With Gingersnap Crumb Crust


In [101]:
count_name = CountVectorizer()

count_matrix_name = count_name.fit_transform(df2['soup_name'])


In [102]:
indices = pd.Series(df2.index, index=df2['Name'])

In [103]:
def show_best_results(df, scores_array, top_n=1):
  sorted_indices = scores_array.argsort()[::-1]
  for position, idx in enumerate(sorted_indices[:top_n]):
    row = df.iloc[idx]
    food_name = row["Name"]
    score = scores_array[idx]
    return food_name

def query_food_name(query):
    query_vectorized = count_name.transform([query])
    scores = query_vectorized.dot(count_matrix_name.transpose())
    scores_array = scores.toarray()[0]
    return show_best_results(df2, scores_array)

In [104]:
query = "fried chicken"
query_vectorized = count_name.transform([query])

scores = query_vectorized.dot(count_matrix_name.transpose())
scores_array = scores.toarray()[0]
show_best_results(df2, scores_array)

'Japanese Chicken Karaage (Fried Chicken)'

In [105]:
def get_recommendations(food_name, cosine_sim=cosine_sim):
    
    matched_food_name = query_food_name(food_name)
    idx = indices[matched_food_name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    if len(sim_scores) > 99:
        sim_scores = sim_scores[1:50]
    else:
        sim_scores =  sim_scores[1:len(sim_scores)]
        
    food_indices = [i[0] for i in sim_scores]

    return df2.iloc[food_indices]

In [106]:
get_recommendations("fried chicken")

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,soup,soup_name
1614,3929,Oven Fried Chicken II,1886,HELEN PEAGRAM,PT40M,PT10M,PT50M,1999-11-03T07:10:00Z,Make and share this Oven Fried Chicken II reci...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,623.0,12.3,0.6,0.1,15.3,4.0,NaN,"c(""Preheat oven to 450"", ""Combine all dry ingr...",chicken parts flour salt fresh pepper paprika ...,Oven Fried Chicken II
2206,4941,Katherine's Fried Chicken,1534,Tonkcats,PT1H30M,PT0S,PT1H30M,1999-12-01T20:03:00Z,Make and share this Katherine's Fried Chicken ...,character(0),...,84.0,35.2,6.1,0.7,28.1,8.0,NaN,"c(""In a large bowl add flour, pepper, seasoned...",whole wheat flour black pepper seasoning salt ...,Katherine's Fried Chicken
114,236,Chicken Curry II,1597,Kathy Bloor,NaN,PT1H,PT1H,1999-08-12T06:14:00Z,"This is a very mild chicken curry, very suitab...",character(0),...,544.8,17.3,1.0,3.5,26.1,8.0,NaN,"c(""The day before, place whole chicken in a la...",chicken unsalted butter onions salt flour curr...,Chicken Curry II
8671,15740,Pecan-Crusted Chicken Drumsticks,20754,RecipeNut,PT45M,PT15M,PT1H,2001-12-13T09:25:00Z,Make and share this Pecan-Crusted Chicken Drum...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,947.8,7.3,0.8,2.9,60.5,6.0,NaN,"c(""PREHEAT oven to 400°E Toss chicken legs wit...",chicken drumsticks chicken legs egg pecans sug...,Pecan-Crusted Chicken Drumsticks
8554,15583,Crispy Coated Garlic Chicken,21752,Mirj2338,PT1H30M,PT8H,PT9H30M,2001-12-11T11:22:00Z,Very delicious and garlicky! Share this with s...,character(0),...,370.6,8.8,0.5,0.8,85.4,8.0,NaN,"c(""Place chicken in a large, shallow container...",chicken olive oil garlic garlic powder salt pe...,Crispy Coated Garlic Chicken
1958,4510,Fried Chicken Livers II,1639,elt8588,NaN,PT0S,PT0S,1999-11-15T04:26:00Z,Make and share this Fried Chicken Livers II re...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3203.7,108.2,3.4,0.7,104.2,NaN,1 batch,"c(""Mix milk and egg; set aside. Mix dry ingre...",chicken liver milk egg baking powder salt flou...,Fried Chicken Livers II
3754,8350,Crispy Chicken Drumsticks,82616,Shirl J 831,PT1H,PT10M,PT1H10M,2000-03-16T20:09:00Z,Make and share this Crispy Chicken Drumsticks ...,character(0),...,1020.6,32.5,2.2,2.7,37.8,4.0,NaN,"c(""Rinse chicken with cold water, and pat dry....",chicken drumsticks parmesan cheese garlic powd...,Crispy Chicken Drumsticks
4044,8782,"Roast ""Sticky"" Chicken",6411,C Chatwin,PT5H10M,PT15M,PT5H25M,2001-04-20T18:06:00Z,"Beautiful and delicious, this incredibly moist...","c(""https://img.sndimg.com/food/image/upload/w_...",...,1619.9,6.2,1.4,2.3,17.6,6.0,2 Servings per Lb,"c(""(Day 1) Blend all spices together and set a...",salt paprika cayenne pepper onion powder thyme...,"Roast ""Sticky"" Chicken"
1554,3842,Real Fried Chicken,1859,Robb114,PT2H,PT20M,PT2H20M,1999-11-01T07:29:00Z,There's only one right way to fry chicken and ...,character(0),...,4811.0,47.9,1.8,0.2,48.7,NaN,NaN,"c(""In a shallow bowl, combine flour, salt and ...",chicken flour salt pepper butter shortening Wh...,Real Fried Chicken
7053,13440,Easy Chicken Crescents,20893,Brenda,PT15M,PT15M,PT30M,2001-10-29T16:52:00Z,Make and share this Easy Chicken Crescents rec...,character(0),...,1649.1,65.7,4.6,5.9,29.2,6.0,12 crescents (approx),"c(""Preheat oven to 350°F."", ""Combine chicken, ...",cream cheese onion powder garlic powder season...,Easy Chicken Crescents


In [107]:
df2.RecipeCategory.unique()


array(['Frozen Desserts', 'Soy/Tofu', 'Pie', 'Chicken', 'Vegetable',
       'Chicken Breast', 'Dessert', 'Beverages', 'Southwestern U.S.',
       'Stew', 'Lactose Free', 'Weeknight', 'Yeast Breads', 'Cheesecake',
       'Sauces', 'High In...', 'Brazilian', 'Brown Rice', 'Oranges',
       'Free Of...', 'Low Protein', 'Potato', 'Halibut', 'Lamb/Sheep',
       'Breads', 'Spaghetti', 'Lunch/Snacks', 'Beans', 'Very Low Carbs',
       'Pineapple', 'Whole Chicken', 'Low Cholesterol', '< 30 Mins',
       'Chicken Livers', 'Coconut', '< 60 Mins', 'Poultry',
       'Quick Breads', 'Steak', 'Healthy', 'Pork', 'Scones', 'Lobster',
       'Rice', 'Punch Beverage', 'Drop Cookies', 'Spreads', 'Bar Cookie',
       'Crab', 'Pears', 'Cheese', 'Savory Pies', 'Breakfast', 'Chowders',
       'Candy', 'Chutneys', 'White Rice', 'Tex Mex', 'German', 'Meat',
       'Fruit', 'European', 'Smoothies', 'Hungarian', 'Onions',
       'New Zealand', 'Indonesian', 'Lentil', 'Summer', 'Long Grain Rice',
       'Southwe

# Filter Recommendation Results With Constraint Satisfaction Modelling
- memfilter rekomendasi makanan dari content-based filtering degnan csp modelling.
- difilter berdasarkan kandungan nutrisi makanan disesuakian dengan penyakit pengguna,umur,weight, height, dan category makanan
- solve csp model nya pakai optimized backtracking (https://web.stanford.edu/class/cs227/Lectures/lec14.pdf)
- hasilnya berupa meal plan breakfast, lunch, dan dinner

In [114]:
from constraint import *
problem = Problem()

def calorie_intake(w, h, a):
    cal =  10*w + 6.25*h - 5*a +5
    return cal

my_favorite_food = ["Steak", "salad", "fried chicken", "burger", "pizza"]
def get_all_food_recommendations(my_favorite_food):
    foods_recommendation = []
    for myfood in my_favorite_food:
        food_recommendations_from_cbf = get_recommendations(myfood)
        food_recommendations_from_cbf = food_recommendations_from_cbf.to_dict('records')
        for new_food in food_recommendations_from_cbf:
            foods_recommendation.append(new_food)
    return foods_recommendation
    
foods_recommendation = get_all_food_recommendations(my_favorite_food)
random.shuffle(foods_recommendation)
my_calorie_daily_intake = calorie_intake(80, 170, 22)

problem.addVariable("breakfast", foods_recommendation
       
                )

problem.addVariable( "lunch",  foods_recommendation
                )

problem.addVariable( "dinner",  foods_recommendation
                )


def filter_based_on_disease(breakfast, lunch, dinner, calorie_intake, disease, category):
    """
    https://journal.ipb.ac.id/index.php/jtep/article/view/52452/27802
    """
    
    saturated_fat =   breakfast["SaturatedFatContent"] + lunch["SaturatedFatContent"] + dinner["SaturatedFatContent"]
    fat =   breakfast["FatContent"] + lunch["FatContent"] + dinner["FatContent"]
    mufa_pufa = fat - saturated_fat
    
    makanan_bervariasi = breakfast != lunch and lunch != dinner and breakfast  != dinner
    match_category = breakfast["RecipeCategory"] == category["breakfast"] and lunch["RecipeCategory"] == category["lunch"] and dinner["RecipeCategory"] == category["dinner"]
    
    if disease == "diabetes":
        max_fat = breakfast["FatContent"] + lunch["FatContent"] + dinner["FatContent"] < 0.25 * calorie_intake
        max_protein = breakfast["ProteinContent"] + lunch["ProteinContent"] + dinner["ProteinContent"] < 0.15 * calorie_intake
        max_carbo = breakfast["CarbohydrateContent"] + lunch["CarbohydrateContent"] + dinner["CarbohydrateContent"] < 0.60 * calorie_intake
        max_sodium = breakfast["SodiumContent"] + lunch["SodiumContent"] + dinner["SodiumContent"] < 2000
        max_cholesterol = breakfast["CholesterolContent"] + lunch["CholesterolContent"] + dinner["CholesterolContent"] < 200
        max_saturated_fat = saturated_fat < 0.07 * calorie_intake
        if len(category) != 0:
            return max_fat and max_protein and max_carbo and max_sodium and max_cholesterol and max_saturated_fat  and makanan_bervariasi and match_category
        else:
            return max_fat and max_protein and max_carbo and max_sodium and max_cholesterol and max_saturated_fat  and makanan_bervariasi
        
    elif disease == "cardiovascular":
        max_fat = breakfast["FatContent"] + lunch["FatContent"] + dinner["FatContent"] < 0.25 * calorie_intake
        max_protein =  breakfast["ProteinContent"] + lunch["ProteinContent"] + dinner["ProteinContent"] < 0.15 * calorie_intake
        max_carbo = breakfast["CarbohydrateContent"] + lunch["CarbohydrateContent"] + dinner["CarbohydrateContent"] < 0.60 * calorie_intake
        max_sodium = breakfast["SodiumContent"] + lunch["SodiumContent"] + dinner["SodiumContent"] < 2000
        max_cholesterol = breakfast["CholesterolContent"] + lunch["CholesterolContent"] + dinner["CholesterolContent"] < 200
        max_saturated_fat = saturated_fat < 0.07 * calorie_intake
        if len(category) != 0:
            return max_fat and max_protein and max_carbo and max_sodium and max_cholesterol and max_saturated_fat and makanan_bervariasi and match_category
        else:
            return max_fat and max_protein and max_carbo and max_sodium and max_cholesterol and max_saturated_fat and makanan_bervariasi

    elif disease == "hypertension":
        max_fat =  breakfast["FatContent"] + lunch["FatContent"] + dinner["FatContent"] < 0.25 * calorie_intake
        max_protein = breakfast["ProteinContent"] + lunch["ProteinContent"] + dinner["ProteinContent"] < 0.15 * calorie_intake
        max_carbo = breakfast["CarbohydrateContent"] + lunch["CarbohydrateContent"] + dinner["CarbohydrateContent"]  < 0.60 * calorie_intake
        max_sodium =  breakfast["SodiumContent"] + lunch["SodiumContent"] + dinner["SodiumContent"]< 1000
        max_saturated_fat = saturated_fat < 0.07 * calorie_intake
        if len(category) != 0:
            return max_fat and max_protein and max_carbo and max_sodium and max_saturated_fat and makanan_bervariasi and match_category
        else:
            return max_fat and max_protein and max_carbo and max_sodium and max_saturated_fat and makanan_bervariasi
 
    elif disease == "normal":
        max_fat = breakfast["FatContent"] + lunch["FatContent"] + dinner["FatContent"] < 0.25 * calorie_intake
        max_protein =  breakfast["ProteinContent"] + lunch["ProteinContent"] + dinner["ProteinContent"] < 0.15 * calorie_intake
        max_carbo = breakfast["CarbohydrateContent"] + lunch["CarbohydrateContent"] + dinner["CarbohydrateContent"]  < 0.60 * calorie_intake
        max_sodium = breakfast["SodiumContent"] + lunch["SodiumContent"] + dinner["SodiumContent"] < 2000
        if category != "": 
            return max_fat and max_protein and max_carbo and max_sodium and makanan_bervariasi  and match_category
        else:
            return max_fat and max_protein and max_carbo and max_sodium and makanan_bervariasi
        
        

problem.addConstraint(lambda breakfast, lunch, dinner: filter_based_on_disease(breakfast, lunch, dinner, my_calorie_daily_intake, "normal", 
                                                                              {"breakfast": "Vegetable", "lunch": "Chicken Breast", "dinner": "Steak"}
                                                                              ),
                       ("breakfast", "lunch", "dinner"))


meal_plan_recommendation = problem.getSolution()
print("breakfast: ", meal_plan_recommendation["breakfast"], " \n")
print("lunch: ", meal_plan_recommendation["lunch"], " \n")
print("dinner: ", meal_plan_recommendation["dinner"], " \n")


breakfast:  {'RecipeId': 13232, 'Name': 'Stuffed Cabbage with Cranberry Sauce', 'AuthorId': 21752, 'AuthorName': 'Mirj2338', 'CookTime': 'PT2H15M', 'PrepTime': 'PT30H', 'TotalTime': 'PT32H15M', 'DatePublished': '2001-10-25T10:35:00Z', 'Description': "This is from one of my favorite cookbook authors, Joan Nathan. It's a great combination of Jewish and American cuising, perfect for Thanksgiving or Sukkot", 'Images': '"https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/13/23/2/picILhElJ.jpg"', 'RecipeCategory': 'Vegetable', 'Keywords': 'c("Meat", "European", "Kosher", "Free Of...", "Thanksgiving", "Weeknight")', 'RecipeIngredientQuantities': 'c("1", "1", "1 1/2", NA, "1/4", "1/2", "1/2", "1", "1", "2", "1/2", NA, NA, "1", "1", "4")', 'RecipeIngredientParts': 'jellied cranberry sauce tomato sauce water brown sugar raisins fresh cranberries apple head of cabbage ground beef rice salt fresh ground pepper egg onion ketchup', 'AggregatedRating': 5.0, 

In [115]:
my_favorite_food = [ "salad"]
def get_all_food_recommendations(my_favorite_food):
    foods_recommendation = []
    for myfood in my_favorite_food:
        food_recommendations_from_cbf = get_recommendations(myfood)
        food_recommendations_from_cbf = food_recommendations_from_cbf.to_dict('records')
        for new_food in food_recommendations_from_cbf:
            foods_recommendation.append(new_food)
    return foods_recommendation


foods_recommendation = get_all_food_recommendations(my_favorite_food)
print(len(foods_recommendation))
print(foods_recommendation)

for food in foods_recommendation:
    if food["RecipeCategory"] == "Vegetable":
        print(food)

49
[{'RecipeId': 8969, 'Name': 'Frijoles Negros (Cuban Black Beans)', 'AuthorId': 9121, 'AuthorName': 'Carol Bullock', 'CookTime': 'PT45M', 'PrepTime': 'PT15M', 'TotalTime': 'PT1H', 'DatePublished': '2001-05-11T09:41:00Z', 'Description': 'Make and share this Frijoles Negros (Cuban Black Beans) recipe from Food.com.', 'Images': 'c("https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/89/69/01464300339.jpeg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/89/69/picYj3Bm1.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/89/69/picsmLLNx.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/89/69/picJ2pV9s.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/89/69/picjVZJhT.jpg", \n"https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_9

In [116]:
# save vectorizer
with open('recommendation_cosine_sim.pkl', 'wb') as file:
    pickle.dump(cosine_sim, file)


with open('search_count_vec.pkl', 'wb') as file:
    pickle.dump(count_name, file)

with open('count_matrix_name.pkl', 'wb') as file:
    pickle.dump(count_matrix_name, file)